In [16]:
using LowLevelFEM
import LowLevelFEM as FEM

gmsh.initialize()

In [17]:
gmsh.open("cylinder.geo")

Info    : Reading 'cylinder.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 40%] Meshing curve 2 (Line)
Info    : [ 70%] Meshing curve 3 (Circle)
Info    : Done meshing 1D (Wall 0.0004648s, CPU 0.000464s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Cylinder, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0452821s, CPU 0.035438s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume with 1 connected component
Info    : Tetrahedrizing 351 nodes...
Info    : Done tetrahedrizing 359 nodes (Wall 0.00705857s, CPU 0.006355s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.0162867s, CPU 0.015434s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 2.08088 

In [18]:
mat = FEM.material("body", ν=0.3)
problem = FEM.Problem([mat])

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 1443)


LowLevelFEM.Problem("cylinder", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5)], 1.0, 8030)

In [19]:
n1x(x, y, z) = x
n1y(x, y, z) = y
n2x(x, y, z) = -y
n2y = n1x
cs = FEM.CoordinateSystem([n1x, n1y, 0], [n2x, n2y, 0])
Q = FEM.rotateNodes(problem, "body", cs);

In [20]:
supp = FEM.displacementConstraint("left", uy=0, uz=0)

("left", 0 + 1im, 0, 0)

In [21]:
fx(x, y, z) = -y
fy(x, y, z) = x
load = FEM.load("right", fx=fx, fy=fy, fz=3)

("right", Main.fx, Main.fy, 3)

In [22]:
f2x(x, y, z) = -y * z / 100000 - 0.3 * x / 10000
f2y(x, y, z) = x * z / 100000 - 0.3 * y / 10000
f2z(x, y, z) = 0.01 / 100 * z
torque = FEM.field("body", fx=f2x, fy=f2y, fz=f2z)
q = FEM.vectorField(problem, [torque]);

In [23]:
f1(x, y, z) = y
f2 = FEM.field("right", f=f1)
qq = FEM.scalarField(problem, [f2])
qqq = FEM.showDoFResults(problem, qq, :scalar)

0

In [24]:
S = FEM.solveStress(problem, q)
S = FEM.elementsToNodes(problem, S);

In [25]:
q1 = Q' * q
S1 = FEM.solveStress(problem, q)
S1 = FEM.elementsToNodes(problem, S1);
S1 = Q' * S1 * Q;

In [26]:
u = FEM.showDoFResults(problem, q, :uvec)
ux = FEM.showDoFResults(problem, q, :ux)
uy = FEM.showDoFResults(problem, q, :uy)
uz = FEM.showDoFResults(problem, q, :uz)
sx = FEM.showDoFResults(problem, S, :sx)
sy = FEM.showDoFResults(problem, S, :sy)
sz = FEM.showDoFResults(problem, S, :sz)
sxy = FEM.showDoFResults(problem, S, :sxy)
syz = FEM.showDoFResults(problem, S, :syz, visible=true)
szx = FEM.showDoFResults(problem, S, :szx)

10

In [27]:
u1 = FEM.showDoFResults(problem, q1, :uvec, name="uvec-HKR")
u1x = FEM.showDoFResults(problem, q1, :ux, name="ux-HKR")
u1y = FEM.showDoFResults(problem, q1, :uy, name="uy-HKR")
u1z = FEM.showDoFResults(problem, q1, :uz, name="uz-HKR")
s1x = FEM.showDoFResults(problem, S1, :sx, name="sx-HKR")
s1y = FEM.showDoFResults(problem, S1, :sy, name="sy-HKR")
s1z = FEM.showDoFResults(problem, S1, :sz, name="sz-HKR")
s1xy = FEM.showDoFResults(problem, S1, :sxy, name="sxy-HKR")
s1yz = FEM.showDoFResults(problem, S1, :syz, name="syz-HKR")
s1zx = FEM.showDoFResults(problem, S1, :szx, name="szx-HKR")

20

In [28]:
p1 = FEM.plotOnPath(problem, "path1", s1yz)
p2 = FEM.plotOnPath(problem, "path2", s1yz)

22

In [29]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [30]:
gmsh.finalize()